In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：用于批量预测的AutoML训练表格预测模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_forecasting_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_forecasting_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_tabular_forecasting_batch.ipynb">
      在 Google Cloud Notebooks 中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex SDK创建表格式预测模型，并使用Google Cloud [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users) 模型进行批量预测。

### 数据集

本教程使用的数据集是BigQuery公共数据集，[纽约时报美国冠状病毒数据库](https://pantheon.corp.google.com/marketplace/product/the-new-york-times/covid19_us_cases)。该数据集不需要任何特征工程。在本教程中使用的数据集版本存储在公共Cloud Storage桶中。

### 目标

在本教程中，您将从Python脚本创建一个AutoML表格式预测模型，然后使用Vertex SDK进行批量预测。您也可以选择使用`gcloud`命令行工具或在Cloud控制台上线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

使用批量预测和在线预测之间有一个关键区别：

* 预测服务：对整个实例集合（即一个或多个数据项）进行按需预测，并实时返回结果。

* 批量预测服务：对整个实例集合进行排队（批量）预测，并在准备就绪时将结果存储在Cloud存储存储桶中。

### 成本

本教程使用了Google Cloud的可计费组件：

* Vertex AI
* 云存储

了解[Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing)和[云存储
价格](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用情况生成成本估算。

### 设置本地开发环境

如果您正在使用Colab或Google Cloud Notebook，您的环境已经满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

[设置Python开发环境的Cloud Storage指南](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了简要的说明：

1. [安装和初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)，并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端窗口的命令行中运行 `pip3 install jupyter`。

5. 要启动Jupyter，请在终端窗口的命令行中运行 `jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

安装

安装最新版本的Python的Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### 重新启动内核

安装完附加包之后，您需要重新启动笔记本内核，以便它能找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

该教程不需要 GPU 运行时。

### 设置您的 Google Cloud 项目

**无论您使用的是哪种笔记本环境，下面的步骤都是必须的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。首次创建帐户时，您将获得 $300 的免费信用额度，用于支付计算/存储成本。

2. [确保为您的项目启用了计费功能。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、Compute Engine API 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行这个笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 会将以 `!` 开头的行作为 shell 命令运行，并且会插入以 `$` 开头的 Python 变量。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于整个笔记本的操作。以下是Vertex AI支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能在Vertex AI中使用多区域存储桶进行训练。并非所有区域都支持所有Vertex AI服务。

了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能正在使用共享测试账户或项目。为了避免不同用户在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称中。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google 云帐户

**如果您正在使用 Google 云笔记本**，您的环境已经经过验证。跳过这一步。

**如果您正在使用 Colab**，运行下面的单元格，并按提示进行身份验证，通过 oAuth 认证您的帐户。

**否则**，请按照以下步骤操作：

在 Cloud 控制台中，转到 [创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务帐号**。

在 **服务帐号名称** 字段中输入名称，然后点击 **创建**。

在 **授予此服务帐号对项目的访问权限** 部分，点击角色下拉列表。在筛选框中输入 "Vertex"，选择 **Vertex 管理员**。在筛选框中输入 "存储对象管理员"，选择 **存储对象管理员**。

点击创建。包含您密钥的 JSON 文件将下载到您的本地环境中。

在下面的单元格中将您的服务帐号密钥路径作为 GOOGLE_APPLICATION_CREDENTIALS 变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，以下步骤都是必须的。**

当您初始化Python的Vertex SDK时，您需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源中的所有数据在不同会话中保留的地方。

请在下方设置您的云存储桶的名称。存储桶的名称必须在所有Google Cloud项目中全局唯一，包括您组织以外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶中的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

初始化Python环境下的Vertex SDK

为您的项目和相应的存储桶初始化Python环境下的Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

# 教程

现在您已经准备好开始创建您自己的AutoML表格预测模型。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/ai-platform/covid/bigquery-public-covid-nyt-us-counties-train.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的纽约时报COVID数据库数据集的一个版本，使用CSV索引文件。

首先要快速查看数据。通过计算CSV索引文件中的行数 (`wc -l`) 来计算示例的数量，然后查看前几行。

您还需要了解标签列的标题名称以进行训练，该名称保存在 `label_column` 中。对于此数据集，它是CSV文件中的最后一列。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = str(heading).split(",")[-1].split("'")[0]
print("Label Column Name", label_column)
if label_column is None:
    raise Exception("label column missing")

### 创建数据集

接下来，使用`TimeSeriesDataset`类的`create`方法创建`Dataset`资源，该方法需要以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `bq_source`：或者，从BigQuery表中导入数据项到`Dataset`资源中。

这个操作可能需要几分钟。

In [ ]:
dataset = aip.TimeSeriesDataset.create(
    display_name="NY Times COVID Database" + "_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
)

label_column = "mean_temp"
time_column = "date"
time_series_id_column = "county"

print(dataset.resource_name)

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "date"}},
    {"auto": {"column_name": "state_name"}},
    {"auto": {"column_name": "county_fips_code"}},
    {"auto": {"column_name": "confirmed_cases"}},
    {"auto": {"column_name": "deaths"}},
]

label_column = "deaths"
time_column = "date"
time_series_identifier_column = "county"

### 创建并运行训练管道

为了训练一个AutoML模型，您需要执行两个步骤：1) 创建一个训练管道，2) 运行这个管道。

#### 创建训练管道

使用`AutoMLForecastingTrainingJob`类创建一个AutoML训练管道，包括以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `column_transformations`：（可选）要应用于输入列的转换。
- `optimization_objective`：最小化或最大化的优化目标。
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

实例化的对象是训练管道的DAG（有向无环图）。

In [ ]:
dag = aip.AutoMLForecastingTrainingJob(
    display_name="train-iowa-liquor-sales-automl_1",
    optimization_objective="minimize-rmse",
    column_transformations=TRANSFORMATIONS,
)

#### 运行训练管道

接下来，您可以运行DAG来启动训练作业，调用方法`run`，并使用以下参数：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：已训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `target_column`：作为标签进行训练的列名。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 =小时）。
- `time_column`：
- `time_series_identifier_column`：

完成后，`run`方法将返回`Model`资源。

执行训练管道将耗时最多20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    target_column=label_column,
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    available_at_forecast_columns=[time_column],
    unavailable_at_forecast_columns=[label_column],
    time_series_attribute_columns=["state_name", "county_fips_code", "confirmed_cases"],
    forecast_horizon=30,
    # context_window=30,
    data_granularity_unit="day",
    data_granularity_count=1,
    weight_column=None,
    budget_milli_node_hours=1000,
    model_display_name="covid_" + TIMESTAMP,
    predefined_split_column_name=None,
)

## 评估模型评分
在您的模型训练完成后，您可以查看其评估分数。

首先，您需要获取对新模型的引用。与数据集一样，您可以使用在部署模型时创建的模型变量的引用，或者您可以列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=covid_" + TIMESTAMP)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

发送一个批量预测请求

发送一个批量预测到您部署的模型。

制作测试项目

您将使用合成数据作为测试数据项目。不必担心我们使用合成数据 - 我们只是想演示如何进行预测。

In [ ]:
HEADING = "date,county,state_name,county_fips_code,confirmed_cases,deaths"
INSTANCE_1 = "2020-10-13,Adair,Iowa,19001,103,null"
INSTANCE_2 = "2020-10-29,Adair,Iowa,19001,197,null"

创建批量输入文件

现在创建一个批量输入文件，将其存储在您的本地云存储存储桶中。与图像、视频和文本不同，表格的批量输入文件只支持CSV格式。对于CSV文件，您需要：

- 第一行是带有特征（字段）标题名称的标题。
- 每个剩余行是一个单独的预测请求，具有相应的特征值。

例如：

    "feature_1", "feature_2", ...
    value_1, value_2, ...

In [ ]:
import tensorflow as tf

gcs_input_uri = BUCKET_NAME + "/test.csv"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    f.write(HEADING + "\n")
    f.write(str(INSTANCE_1) + "\n")
    f.write(str(INSTANCE_2) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 发起批量预测请求

现在您的模型资源已经训练好了，您可以通过调用batch_predict()方法来发起批量预测，使用以下参数：

- `job_display_name`：批量预测作业的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的 Cloud Storage 位置。
- `instances_format`：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `sync`：如果设置为True，则调用将在等待异步批处理作业完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="covid_" + TIMESTAMP,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_NAME,
    instances_format="csv",
    predictions_format="csv",
    sync=False,
)

print(batch_predict_job)

等待批量预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测作业完成。

In [ ]:
batch_predict_job.wait()

获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的Cloud Storage输出存储桶中。您可以调用iter_outputs()方法来获取生成的结果文件列表。每个文件都以CSV格式包含一个或多个预测请求：

- CSV表头 + 预测标签
- CSV行 + 预测值，每个预测请求一个

In [ ]:
import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            print(line)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于此教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- AutoML训练作业
- 批处理作业
- 定制作业
- 超参数调整作业
- Cloud Storage存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME